I found most of the streaming portion of this at http://adilmoujahid.com/posts/2014/07/twitter-analytics/

This file below is saved as "twitter_streaming.py" in my folder.

In [ ]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

access_token = "-"
access_token_secret = "-"
consumer_key = "-"
consumer_secret = "-"


#This is a basic listener that just prints received tweets to stdout.
class StdOutListener(StreamListener):

    def on_data(self, data):
        print(data)
        return True

    def on_error(self, status):
        print(status)


if __name__ == '__main__':

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l)

    #This line filter Twitter Streams to capture data by the keywords:
    stream.filter(track=['#kateswall', '#katesteinle', '#steinle'])

Here, I run the command "python twitter_streaming.py > twitter_data.txt" in the terminal to run the above file and save the output to a new text file called "twitter_data.txt".

In [ ]:
import json

tweets_data_path = '/Users/Marty/Documents/BMKT670/TwitterAPI/twitter_data.txt'

tweets_data = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue
        
print(len(tweets_data))

The following is my own custom code that implements the spaCy library to run analysis of the Kate Steinel tweets.

In [171]:
#spacy is a framework that allows
#deep dives into nlp in a variety of ways
import spacy
#This is a HUGE lexicon of words
nlp = spacy.load('en_core_web_lg')

In [172]:
#initialize tweets list we will pull from json
tweets_list = []
with open('/Users/Marty/Documents/BMKT670/TwitterAPI/twitter_data.txt', 'r') as f:
    for line in f:
        try:
            #json saved as huge dictionary of dictionaries
            #this was very hard to figure out
            tweet = json.loads(line)
            tweets_list.append(tweet)
        except:
            #catches any key errors we might run into
            KeyError
    #print(json.dumps(tweet, indent=4))

In [12]:
final_tweet_list = []
for tweet in tweets_list:
    #check if tweet is a retweet or not
    #we want to pull original tweets first
    if "retweeted_status" not in tweet:
        #adds UserName, # of followers and friends, tweet text, and nlp version of tweet
        temp_list = [tweet["user"]["name"], tweet["user"]["followers_count"], 
                     tweet["user"]["friends_count"], str(tweet["text"]), nlp(tweet["text"])]
        final_tweet_list.append(temp_list)

In [13]:
print(len(final_tweet_list))

1942


The following cell does most of the heavy lifting for this project.
It runs through the tweet list, and creates a list that I can use for further analysis.

In [188]:
counter = 0

#List and set for tweets with similarity > .95
sim_list = []
sim_set = set()
#List and set for tweets with similarity between .9 and .95
sim_list_2 = []
sim_set_2 = set()
for tweet in final_tweet_list:
    for other_tweet in final_tweet_list:
        #Find the similarity score between each tweet in the list
        if (tweet[3] != other_tweet[3] and tweet[4].similarity(other_tweet[4]) > .95):
            counter += 1
            #This code creates the parts of speech for each tweet and other tweet
            tweet_pos = " ".join([word.pos_ for word in tweet[4]])
            other_tweet_pos = " ".join([word.pos_ for word in other_tweet[4]])
            #Build the temporary list to add to the main list
                #Maybe not best practice?
            temp_list = [tweet[0:4], tweet_pos, other_tweet[0:4], 
                             other_tweet_pos, tweet[4].similarity(other_tweet[4])]
            sim_list.append(temp_list)
            sim_set.add(tweet[3])
            sim_set.add(tweet[4])
        #Find slightly looser similarity score and repeat above
        elif (tweet[3] != other_tweet[3] and .95 > tweet[4].similarity(other_tweet[4]) > .90):
            counter += 1
            tweet_pos = " ".join([word.pos_ for word in tweet[4]])
            other_tweet_pos = " ".join([word.pos_ for word in other_tweet[4]])
            temp_list = [tweet[0:4], tweet_pos, other_tweet[0:4], 
                             other_tweet_pos, tweet[4].similarity(other_tweet[4])]
            sim_list_2.append(temp_list)
            sim_set_2.add(tweet[3])
            sim_set_2.add(tweet[4])
            #Counter to check progress
            if counter % 1000 == 0:
                print(counter)

1000
2000
3000
4000
5000
6000
7000
8000
9000
11000
12000
13000
14000
15000
16000
17000
18000
21000
23000
24000
25000
26000
27000
28000
29000
31000
32000
33000
34000
35000
36000
39000
40000
41000
43000
44000
45000
46000
47000
48000
49000
52000
53000
54000
55000
56000
57000
58000
60000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
77000
78000
79000
80000
81000
82000
83000
84000
85000
87000
88000
89000
92000
93000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
108000
112000
113000
114000
115000
120000
121000
122000
123000
124000
128000
130000
132000
133000
134000
135000
136000
137000
138000
139000
141000
143000
145000
146000
149000
151000
152000
153000


In [247]:
#Displacy displays spacy results in color-coded sequence for similarity score > .95
from spacy import displacy
from random import randint
#to randomly choose one set of tweets
randnum = randint(0,1500)
#Print out info for each tweet to compare
print("Name: " + str(sim_list[randnum][0][0]))
print("Followers: " + str(sim_list[randnum][0][1]))
print("Friends: " + str(sim_list[randnum][0][2]))
print("Parts of Speech: " + str(sim_list[randnum][1]))
displacy.render(nlp(sim_list[randnum][0][3]), style='ent', jupyter= True)
print("Name: " + str(sim_list[randnum][2][0]))
print("Followers: " + str(sim_list[randnum][2][1]))
print("Friends: " + str(sim_list[randnum][2][2]))
print("Parts of Speech: " + str(sim_list[randnum][3]))
displacy.render(nlp(sim_list[randnum][2][3]), style='ent', jupyter= True)

print("Similarity Score: " + str(sim_list[randnum][4]))

Name: Zod Thanátos
Followers: 32
Friends: 99
Parts of Speech: PUNCT NOUN SYM PROPN SPACE SYM PROPN X


Name: OneCoast
Followers: 430
Friends: 1012
Parts of Speech: VERB SYM VERB SYM PROPN


Similarity Score: 0.999999957539


In [239]:
#Displacy displays spacy results in color-coded sequence for similarity score between .90 and .95
randnum = randint(0,1500)
#Print out info for each tweet to compare
print("Name: " + str(sim_list_2[randnum][0][0]))
print("Followers: " + str(sim_list_2[randnum][0][1]))
print("Friends: " + str(sim_list_2[randnum][0][2]))
print("Parts of Speech: " + str(sim_list_2[randnum][1]))
displacy.render(nlp(sim_list_2[randnum][0][3]), style='ent', jupyter= True)
print("Name: " + str(sim_list_2[randnum][2][0]))
print("Followers: " + str(sim_list_2[randnum][2][1]))
print("Friends: " + str(sim_list_2[randnum][2][2]))
print("Parts of Speech: " + str(sim_list_2[randnum][3]))
displacy.render(nlp(sim_list_2[randnum][2][3]), style='ent', jupyter= True)

print("Similarity Score: " + str(sim_list_2[randnum][4]))

Name: America Returns
Followers: 6685
Friends: 6306
Parts of Speech: PROPN NOUN VERB PROPN NOUN PART VERB DET VERB NOUN PUNCT PRON VERB VERB PART VERB PRON ADP DET PRON PUNCT PRON


Name: Defend Liberty
Followers: 13372
Friends: 10843
Parts of Speech: PROPN PROPN PRON VERB PART VERB DET NOUN ADP DET PUNCT PROPN PUNCT PROPN ADP PROPN CCONJ PRON VERB ADV VERB ADV VERB PROPN NOUN PUNCT


Similarity Score: 0.906401580897
